In [66]:
import pandas as pd
import numpy as np
import pandas as pd
import biogeme.database as db
import biogeme.biogeme as bio
from biogeme import models, tools
from biogeme.expressions import Beta, Variable, bioDraws, MonteCarlo, log, Power, exp, Derive
import scipy.stats as st

Loading Data and creating new variable

In [83]:
# Read the CSV file
df = pd.read_csv(r"C:\Users\alexi\Desktop\Semester Project GIT\Semester-project\Data\apollo_modeChoiceData.csv")
df


,ID,RP,SP,RP_journey,SP_task,av_car,av_bus,av_air,av_rail,time_car,...,access_air,service_air,time_rail,cost_rail,access_rail,service_rail,female,business,income,choice
0,1,1,0,1.0,NaN,0,0,1,1,0,...,55,0,140,55,5,0,0,0,46705,4
1,1,1,0,2.0,NaN,0,0,1,1,0,...,45,0,170,45,20,0,0,0,46705,4
2,1,0,1,NaN,1.0,0,0,1,1,0,...,55,3,170,35,5,2,0,0,46705,4
3,1,0,1,NaN,2.0,0,0,1,1,0,...,45,1,120,75,5,3,0,0,46705,4
4,1,0,1,NaN,3.0,0,0,1,1,0,...,40,1,155,75,25,2,0,0,46705,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7995,500,0,1,NaN,10.0,1,1,0,1,300,...,0,0,170,35,5,3,0,1,19910,4
7996,500,0,1,NaN,11.0,1,1,0,1,275,...,0,0,130,65,15,3,0,1,19910,4
7997,500,0,1,NaN,12.0,1,1,0,1,345,...,0,0,140,75,5,2,0,1,19910,1
7998,500,0,1,NaN,13.0,1,1,0,1,250,...,0,0,130,65,10,2,0,1,19910,4


In [148]:
#create new variable mean income:
df['mean_income'] =np.mean(df['income'])

#fill nAn values with 0 (same as in Apollo)
df = df.fillna(0) 

#creating biogeme database

database = db.Database('SP_RP_model', df)
globals().update(database.variables) #transform all columns into variables


Defining Parameters

In [133]:
asc_car = Beta('asc_car', 0, None, None, 1) #fixed parameter, to 0
asc_bus = Beta('asc_bus', 0, None, None, 0)
asc_air = Beta('asc_air', 0, None, None, 0)
asc_rail = Beta('asc_rail', 0, None, None, 0)

asc_bus_shift_female = Beta('asc_bus_shift_female', 0, None, None, 0)
asc_air_shift_female = Beta('asc_air_shift_female', 0, None, None, 0)
asc_rail_shift_female = Beta('asc_rail_shift_female', 0, None, None, 0)

b_tt_car = Beta('b_tt_car', 0, None, None, 0)
b_tt_bus = Beta('b_tt_bus', 0, None, None, 0)
b_tt_air = Beta('b_tt_air', 0, None, None, 0)
b_tt_rail = Beta('b_tt_rail', 0, None, None, 0)
b_tt_shift_business = Beta('b_tt_shift_business', 0, None, None, 0)

b_access = Beta('b_access', 0, None, None, 0)

b_cost = Beta('b_cost', 0, None, None, 0)
b_cost_shift_business = Beta('b_cost_shift_business', 0, None, None, 0)

cost_income_elast = Beta('cost_income_elast', 0, None, None, 0)
b_no_frills = Beta('b_no_frills', 0, None, None, 1) #fixed parameter, to 0
b_wifi = Beta('b_wifi', 0, None, None, 0)
b_food = Beta('b_food', 0, None, None, 0)

mu_RP = b_food = Beta('mu_RP', 1, None, None, 1) #fixed parameter, to 1
mu_SP = b_food = Beta('mu_SP', 1, None, None, 0)

### Create alternative specific constants and coefficients using interactions with socio-demographics
asc_bus_value   = asc_bus  + asc_bus_shift_female * female
asc_air_value   = asc_air  + asc_air_shift_female * female
asc_rail_value  = asc_rail + asc_rail_shift_female * female
b_tt_car_value  = b_tt_car + b_tt_shift_business * business
b_tt_bus_value  = b_tt_bus + b_tt_shift_business * business
b_tt_air_value  = b_tt_air + b_tt_shift_business * business
b_tt_rail_value = b_tt_rail + b_tt_shift_business * business
b_cost_value    = ( b_cost +  b_cost_shift_business * business ) * ( income / mean_income ) ** cost_income_elast



Defining the model (utility function)

In [145]:
V_car = asc_car + b_tt_car_value * time_car + b_cost_value * cost_car
V_bus = asc_bus + b_tt_bus_value * time_bus + b_access * access_bus + b_cost_value * cost_bus
V_air = asc_air + b_tt_air_value * time_air + b_access * access_air + b_cost_value * cost_air + b_no_frills * (service_air == 1) + b_wifi * (service_air == 2) + b_food * (service_air == 3)
V_rail = asc_rail + b_tt_rail_value * time_rail + b_access * access_rail + b_cost_value * cost_rail + b_no_frills * (service_rail == 1) + b_wifi * (service_rail == 2) + b_food * (service_rail == 3)

V = {1: V_car, 2: V_bus, 3: V_air, 4: V_rail}
V_RP = {1: mu_RP*V_car, 2: mu_RP*V_bus, 3: mu_RP*V_air, 4: mu_RP*V_rail} #utility for revealed preferences
V_SP = {1: mu_SP*V_car, 2: mu_SP*V_bus, 3: mu_SP*V_air, 4: mu_SP*V_rail} #utility for revealed preferences




# Attach the probabilities for RP part
biogeme_data_RP['prob_RP'] = prob_RP

# Define the logit model for SP
prob_SP = bioLogit(V, av_car=av_car, av_bus=av_bus, av_air=av_air, av_rail=av_rail, choiceVar=choice)

# Filter data for SP
biogeme_data_SP = data[data['SP'] == 1]

# Attach the probabilities for SP part
biogeme_data_SP['prob_SP'] = prob_SP

# Combine models
P = CombineModels(prob_RP, prob_SP)

# Take product across observations for the same individual
P_panel = PanelProd(P)

# Prepare and return outputs
P_prepared = PrepareProb(P_panel)

# Print the results (you might want to adjust this based on Biogeme's conventions)
BIOGEME_OBJECT = {
    "loglike": P_prepared,
    "weight": 1
}

results = estimate(BIOGEME_OBJECT)
print(results.getEstimatedParameters())


NameError: name 'prob_RP' is not defined

In [139]:
# Retrieve the general statistics from the results
general_stats_model_1 = results_model_1.getGeneralStatistics()
print(results_model_1.printGeneralStatistics())

Number of estimated parameters:	14
Sample size:	8000
Excluded observations:	0
Init log likelihood:	-10966.87
Final log likelihood:	-7647.336
Likelihood ratio test for the init. model:	6639.071
Rho-square for the init. model:	0.303
Rho-square-bar for the init. model:	0.301
Akaike Information Criterion:	15322.67
Bayesian Information Criterion:	15420.49
Final gradient norm:	3.0902E-02
Nbr of threads:	8

